# [PyBroMo](http://tritemio.github.io/PyBroMo/) - 1.1 Run the simulation - Single host - chunk

<small>
*This notebook is part of [PyBroMo](http://tritemio.github.io/PyBroMo/) a 
python-based single-molecule Brownian motion diffusion simulator 
that simulates confocal [smFRET](http://en.wikipedia.org/wiki/Single-molecule_FRET)
experiments. You can find the full list of notebooks in 
[Usage Examples](http://tritemio.github.io/PyBroMo/#usage-examples).*
</small>

## Brownian Motion simulation on a single host

Here we will see a simple example usage of the **PyBroMo** software.

The main file is `brownian.py`: it contains the simulator code, 
helper functions and the default simulation parameters.

For convenience, here we use an initialization script that will enter into the **PyBroMo** folder, 
load `brownian.py` and also import some useful IPython functions:

In [ ]:
import pybromo as pbm

In [ ]:
# Simulation time step
t_step = 0.5e-6     # seconds

# Diffusion coefficient
Du = 12.0           # um^2 / s
D = Du*(1e-6)**2

# Time duration of the simulation
t_max = 0.3        # seconds
n_samples = int(t_max/t_step)

# PSF definition
ss = 0.2*1e-6      # lateral dimension (sigma)
psf = pbm.GaussianPSF(xc=0, yc=0, zc=0, sx=ss, sy=ss, sz=3*ss)
psf = pbm.NumericPSF()

# Box definition
box = pbm.Box(x1=-4.e-6, x2=4.e-6, y1=-4.e-6, y2=4.e-6, z1=-6e-6, z2=6e-6)

# Particles definition
p1 = pbm.Particle(x0=-3e-6)
p2 = pbm.Particle(x0=3e-6)
p3 = pbm.Particle(y0=-3e-6)
p4 = pbm.Particle(y0=3e-6)
P = [p1,p2,p3,p4]
P = pbm.gen_particles(15, box)

# Brownian motion and emission simulation
S = pbm.ParticlesSimulation(D=D, t_step=t_step, t_max=t_max,
                       particles=P, box=box, psf=psf)
S.sim_brownian_motion(save_pos=True, total_emission=False)

In [ ]:
pbm.bpl.plot_tracks(S)
pbm.bpl.plot_emission(S)

In [ ]:
# Simulation time step
t_step = 0.5e-6   # seconds

# Time duration of the simulation
t_max = .3 # seconds

# Particles definition
P = pbm.brownian.gen_particles(15, box)

# Particle simulation definition
S = pbm.brownian.ParticlesSimulation(D=D, t_step=t_step, t_max=t_max, particles=P, box=box, psf=psf)

The most important line is the last line which creates an object `S` 
that contains all the simulation parameters (it also contains methods to run 
the simulation). You can print `S` and check the current parameters:

In [ ]:
S

or check the required RAM for the current parameters:

In [ ]:
S.print_RAM()

In [ ]:
S.open_store(chunksize=2**18)

In [ ]:
#S.store.close()

In [ ]:
%%timeit -n1 -r1
S.sim_motion_em_chunk(total_emission=False, delete_pos=True)

In [ ]:
%timeit S.sim_timestamps_em_list(max_rate=3e5, bg_rate=2e3, seed=1)

In [ ]:
print [p.size for p in S.all_times_chunks_list]
print [p.size for p in S.all_par_chunks_list]

In [ ]:
ph_times0 = [ph.copy() for ph in S.all_times_chunks_list]
par0 = [p.copy() for p in S.all_par_chunks_list]

In [ ]:
%timeit S.sim_timestamps_em_list1(max_rate=3e5, bg_rate=2e3, seed=1)

In [ ]:
print [p.size for p in S.all_times_chunks_list]
print [p.size for p in S.all_par_chunks_list]

In [ ]:
ph_times1 = [ph.copy() for ph in S.all_times_chunks_list]
par1 = [p.copy() for p in S.all_par_chunks_list]

In [ ]:
%timeit S.sim_timestamps_em_list2(max_rate=3e5, bg_rate=2e3, seed=1)

In [ ]:
print [p.size for p in S.all_times_chunks_list]
print [p.size for p in S.all_par_chunks_list]

In [ ]:
ph_times2 = [ph.copy() for ph in S.all_times_chunks_list]
par2 = [p.copy() for p in S.all_par_chunks_list]

In [ ]:
print [np.allclose(p1, p2) for p1, p2 in zip(ph_times0, ph_times2)]
print [np.allclose(p1, p2) for p1, p2 in zip(par0, par2)]

In [ ]:
print [np.allclose(p1, p2) for p1, p2 in zip(ph_times0, ph_times1)]
print [np.allclose(p1, p2) for p1, p2 in zip(par0, par1)]

In [ ]:
%timeit -r1 -n1 S.sim_timestamps_em_store(max_rate=3e5, bg_rate=2e3, seed=1, overwrite=True)

In [ ]:
%timeit S.sim_timestamps_em_store(max_rate=3e5, bg_rate=2e3, seed=1, overwrite=True)

In [ ]:
S.timestamps.shape, sum([p.size for p in S.all_times_chunks_list])

In [ ]:
i = 0       
for p in S.all_times_chunks_list:
    print (S.timestamps[i:i+p.size] == p).all(),
    i += p.size

In [ ]:
S2 = load_simulation(S.store_fname)

In [ ]:
S2.store_fname

In [ ]:
S2.store

In [ ]:
S.store.data_file.get_filesize()/1024./1024.

In [ ]:
def merge_timestamps(timestamps_list, kind='quicksort'):
    timestamps = np.hstack(timestamps_list)
    particle = np.hstack([np.ones(ts.size, dtype='uint8') 
                            for ts in timestamps_list])
    argsort = timestamps_p.argsort(kind=kind)
    return timestamps[argsort], particle[argsort]

In [ ]:
from IPython.core.display import HTML
HTML(open("./styles/custom2.css", "r").read())